# Taller Identificación de créditos riesgosos usando SVM

Las entidades financieras desean mejorar sus procedimientos de aprobación de créditos con el fin de disminuir los riesgos de no pago de la deuda, lo que acarrea pérdidas a la entidad. El problema real consiste en poder decidir si se aprueba o no un crédito particular con base en información que puede ser fácilmente recolectada por teléfono o en la web. Se tiene una muestra de 1000 observaciones. Cada registro contiene 20 atributos que recopilan información tanto sobre el crédito como sobre la salud financiera del solicitante. Construya un sistema de recomendación que use máquinas de vectores de soporte.

El archivo de datos se encuentra disponible en el siguiente link:

https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/credit.csv




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline            # Para los Pipelines
from sklearn.preprocessing import StandardScaler # Para la transformacion de los numericos
from sklearn.preprocessing import LabelEncoder   # Para la tranformacion de objects
from sklearn.compose import make_column_selector # Para la tranformacion
from sklearn.compose import ColumnTransformer    # Para la transformacio
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV   # Selecciona los mejores Hiperparametros de un modelo
from sklearn.svm import SVC
from sklearn import svm

# Cargamos el data set
df = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/german.csv"
)
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,residence_history,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,4,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,2,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,3,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,4,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,4,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [2]:
#
# Use el transformador LabelEncoder para preprocesar
# las columnas alfanuméricas del dataframe.
#
# Use los primeros 900 datos para entrenamiento del
# modelo y los 100 restantes para validación.
# 
# Construya el SVM usando los valores por defecto de
# los parámetros.
#
# Compute la matriz de confusión para la muestra de
# validación.
#
# Rta/
# True
# True
# True
# True
#
enc = LabelEncoder()

# Se aplica el codificador a las columnas del dataset
columns = [
    "checking_balance",
    "credit_history",
    "purpose",
    "savings_balance",
    "employment_length",
    "personal_status",
    "other_debtors",
    "property",
    "installment_plan",
    "housing",
    "telephone",
    "foreign_worker",
    "job",
]

for column in columns:
    df[column] = enc.fit_transform(df[column])

# División de la x en Train( primeras 900) y Test (últimas 100)
train_sample = list(range(900))
test_sample = list(range(900, 1000))

# Genera los conjuntos de entrenamiento y prueba
x_train = df.iloc[train_sample, :].copy()
x_test = df.iloc[test_sample, :].copy()

# Se elimina la columna default que corresponde a la variable de salida
x_train.drop("default", axis=1, inplace=True)
x_test.drop("default", axis=1, inplace=True)

# Se genera la variable dependiente
y_train = df.default[train_sample]
y_test = df.default[test_sample]

# Solucion del modelo
modelo_clf_svm = svm.SVC()
modelo_clf_svm.fit(x_train,y_train)
SVC()
y_prediccion = modelo_clf_svm.predict(x_test)
cm1 = confusion_matrix(y_test,y_prediccion)

# ---->>> Evaluación ---->>>
# cm es la matriz de confusion
print(cm1[0][0] == 67)
print(cm1[0][1] == 1)
print(cm1[1][0] == 30)
print(cm1[1][1] == 2)

True
True
True
True


# Punto 2

In [3]:
#
# Encuentre la mejor combinación de kernel y parámetros
# de regularización para los valores suministrados 
# durante el entrenamiento y compute la matriz de 
# confusión para la muestra de prueba.
#
# Rta/
# True
# True
# True
# True
#
kernels = ['rbf', 'linear', 'poly', 'sigmoid']
Cs = [1, 2, 3, 4, 5]

# Creamos la variable de parametros para los pasos del pipeline
estimators = [
    #("transformer_ct", ct),
    ("clf", SVC()),
]

# Creamos el Pipeline invocando los parametros que ya fueron construidos
pipeline = Pipeline(
    steps=estimators,
    verbose=False,
)
#pipeline

# Parametros con el cual se harán las evaluacione y se escográ el mejor resultado
# Se omite el kernel linear porque deteriora el performance del modelo
param_grid = {
        "clf__kernel": ["rbf", "poly", "sigmoid"],        
        "clf__C": [1, 2, 3, 4, 5],
    }

gridSearchCV = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=2,
    n_jobs=-1,
    scoring="accuracy",
    refit=True,
    return_train_score=False,
)

# Entrenamiento
gridSearchCV.fit(x_train, y_train)

# Ejecucion del modelo con los datos de entrenamiento X_train
y_pred = gridSearchCV.predict(x_test)

# Creacion de la matriz de confusión
cm2 = confusion_matrix(y_test,y_pred)

# ---->>> Evaluación ---->>>
# cm2 es la matriz de confusion
print(cm2[0][0] == 68)
print(cm2[0][1] == 0)
print(cm2[1][0] == 30)
print(cm2[1][1] == 2)

True
True
True
True
